In [ ]:
import gc
import torch
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import ORPOConfig, ORPOTrainer, setup_chat_format

In [ ]:
# Flush memory
# del trainer, model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

In [ ]:
base_model = "MaziyarPanahi/calme-2.4-rys-78b"
new_model = "dfurman/CalmeRys-78B-Orpo-v0.1"

In [ ]:
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model, tokenizer = setup_chat_format(model, tokenizer)

# Merge adapter with base model
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

model.push_to_hub(new_model, use_temp_dir=True)
tokenizer.push_to_hub(new_model, use_temp_dir=True)